In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')


[]

In [17]:
from pandas import read_csv
#Load data set
series_influ_A_df = read_csv('../temp_data/influA_vietnam.csv', usecols=[1,2,3], engine='python')
# series_influ_A_df = series_influ_A_df["Influenza A - All types of surveillance"]
series_influ_A_df = series_influ_A_df.dropna()
series_influ_A_df = series_influ_A_df.astype('float32')
series_influ_A_df = series_influ_A_df.values
# normalize the dataset
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(series_influ_A_df)
# Create Training and Test
train = dataset[:735, :]
test = dataset[735:, :]

(792, 3)


In [2]:
import numpy as np
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [3]:
look_back = 2
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [5]:
print(trainX[0])
# reshape input to be [samples, time steps, features]
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


[[0.08536585 0.38735178 0.6863711 ]
 [0.         0.21739128 0.2906404 ]]


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(2, 3)))
model.add(Dense(3))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
732/732 - 2s - 2ms/step - loss: 0.0709
Epoch 2/100
732/732 - 1s - 891us/step - loss: 0.0237
Epoch 3/100
732/732 - 1s - 892us/step - loss: 0.0209
Epoch 4/100
732/732 - 1s - 948us/step - loss: 0.0183
Epoch 5/100
732/732 - 1s - 995us/step - loss: 0.0170
Epoch 6/100
732/732 - 1s - 1ms/step - loss: 0.0163
Epoch 7/100
732/732 - 1s - 877us/step - loss: 0.0161
Epoch 8/100
732/732 - 1s - 882us/step - loss: 0.0160
Epoch 9/100
732/732 - 1s - 899us/step - loss: 0.0159
Epoch 10/100
732/732 - 1s - 955us/step - loss: 0.0159
Epoch 11/100
732/732 - 1s - 910us/step - loss: 0.0159
Epoch 12/100
732/732 - 1s - 874us/step - loss: 0.0159
Epoch 13/100
732/732 - 1s - 890us/step - loss: 0.0159
Epoch 14/100
732/732 - 1s - 876us/step - loss: 0.0158
Epoch 15/100
732/732 - 1s - 886us/step - loss: 0.0158
Epoch 16/100
732/732 - 1s - 949us/step - loss: 0.0158
Epoch 17/100
732/732 - 1s - 896us/step - loss: 0.0157
Epoch 18/100
732/732 - 1s - 872us/step - loss: 0.0158
Epoch 19/100
732/732 - 1s - 876us/step - 

In [13]:
from sklearn.metrics import mean_squared_error
# make predictions
testPredict = model.predict(testX)
# invert predictions
testPredict = scaler.inverse_transform(testPredict)
testY_ = scaler.inverse_transform(testY)
# print(testPredict[:,0], testY_[0])
# calculate root mean squared error
testScore = np.sqrt(mean_squared_error(testY_, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Test Score: 6.81 RMSE


In [18]:
import matplotlib.pyplot as plt
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainX)+(look_back*2)+1:len(dataset)-1, :] = testPredict[0]
plt.plot(scaler.inverse_transform(dataset))
print(testPredictPlot.shape)
plt.plot(testPredictPlot, color = 'green')
plt.show()

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [10]:
# model.save("../model/LSTM.keras")